# 메뉴 crawling

In [31]:
import os
import sys
from selenium import webdriver
import chromedriver_autoinstaller

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import re

In [32]:
def get_menu():
    #크롬드라이버 실행
#     options = webdriver.ChromeOptions()
#     options.add_argument('headless')
    chromedriver_dir = r'C:/Users/LG/Downloads/chromedriver_win32/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir) #, chrome_options=options)
    
    #드라이버에 url을 넣음
    driver.get('https://www.mcdonalds.co.kr/kor/menu/list.do')
    driver.implicitly_wait(3)
    
    #메뉴 더보기 버튼 클릭
    while True:
        try:
            driver.find_element_by_xpath('''//*[@id="btnMore"]/button''').click()
            driver.implicitly_wait(3)
        except:
            break
    
    #BeautifulSoup 사용하여 음식리스트 받아오기        
    soup = BeautifulSoup(driver.page_source,'html.parser')
    menu_list=[]
    list = soup.find(id='menuList').find_all('strong')
    for i in list:
        menu_list.append(i.get_text())
    
    #리스트에서 특수문자 제거
    new_list=[]
    for i in menu_list:
        new_list.append(re.sub('™|®','',i))
        
    #음식 이미지 받아오기
    menu_img=[]
    list2 = soup.find(id='menuList').find_all('img')
    for i in list2:
        menu_img.append('https://www.mcdonalds.co.kr'+i['src'])
        
    #세트 이미지 가져오기
    driver.get('https://www.mcdonalds.co.kr/kor/menu/list.do')
    driver.implicitly_wait(3)
    driver.find_element_by_xpath('''//*[@id="container"]/div[1]/div[2]/div/ul/li[2]/a''').send_keys(Keys.ENTER)
    driver.implicitly_wait(3)

    #메뉴 더보기 버튼 클릭
    while True:
        try:
            driver.find_element_by_xpath('''//*[@id="btnMore"]/button''').click()
            driver.implicitly_wait(3)
        except:
            break

    #BeautifulSoup 사용하여 세트 리스트 받아오기        
    soup = BeautifulSoup(driver.page_source,'html.parser')
    set_img=[]
    list3 = soup.find(id='menuList').find_all('img')
    for i in list3:
        set_img.append('https://www.mcdonalds.co.kr'+i['src'])
    
    result=[]
    i=0
    while i<len(set_img):
        value=[]
        value.append(new_list[i])
        value.append(menu_img[i])
        value.append(set_img[i])
        result.append(value)
        i+=1
        
    return result


# 메뉴정보 DB 업데이트

In [33]:
import pymysql

In [34]:
#수정본
def update_data(input_list):
    #DB연결하여 커서 얻기
    conn = pymysql.connect(host='localhost', user='root', password='0000', db='barcode', charset='utf8')
    cur = conn.cursor() 
    
    #2차원 리스트에서 메뉴명, 이미지, 세트 이미지 리스트 분리
    menulist = []
    imagelist = []
    setlist = []
    for i in input_list:
        menulist.append(i[0])
        imagelist.append(i[1])
        setlist.append(i[2])

    # DB menu 데이터 얻기
    sql = "select * from menu_list"
    cur.execute(sql)

    # fetchall()함수 통해 모든 행 저장
    rows = cur.fetchall();
    DBlist = []

    # 모든 행 중 젤 앞 menu 열만 저장
    for i in rows:
        DBlist.append(i[0])
    del_list = []

    # DB에만 있고 크롤링 리스트에 없는 것 추출
    for i in DBlist:
        if i not in menulist:
            del_list.append(i)

    # 기존 데이터 크롤링한 데이터에 없는 DB 데이터 삭제
    sql = "delete from menu_list where menu = %s"
    for i in range(len(del_list)):
        cur.execute(sql, (del_list[i]))

    # 삽입문(ignore 추가로 중복 무시하고 삽입)
    sql = "insert ignore into menu_list(menu, img, set_img) values(%s, %s, %s)"
    for i in range(len(menulist)):
        cur.execute(sql, (menulist[i], imagelist[i], setlist[i]))

    # DB에 반영
    conn.commit()

    # DB연결 종료
    conn.close()

In [35]:
update_data(get_menu())

# -------------------------------------끝------------------------------------------